In [1]:
# nuclio: ignore
import nuclio

In [ ]:
%%nuclio config 
spec.build.baseImage = "mlrun/mlrun"

In [ ]:
%nuclio config kind="serving"
%nuclio cmd python -m pip install tensorflow==2.2.0
%nuclio cmd python -m pip install keras

In [5]:
import pandas as pd
from mlrun import MLClientCtx
import os
import mlrun
import numpy as np
import tensorflow
import json

In [ ]:
class RNN_Model_Serving(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file,extra_data = self.get_model(suffix = ".h5")
        self.model = tensorflow.keras.models.load_model(model_file)
        
    def predict(self, body):
        try:
            """Generate model predictions from sample."""
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            result = json.dumps(result.tolist())
            return result
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [281]:
# nuclio: end-code

In [ ]:
# test localy

In [ ]:
# Run this to download the pre-trained model to your `models` directory

import os
model_location = 'https://igz-app-lab.s3.us-east-2.amazonaws.com/RNN.h5'
saved_models_directory = os.path.join(os.path.abspath('./'), 'models')

# Create paths
os.makedirs(saved_models_directory, exist_ok=1)
model_filepath = os.path.join(saved_models_directory, os.path.basename(model_location))
!wget -nc -P {saved_models_directory} {model_location}




model_dir = model_filepath
model_server = RNN_Model_Serving(model_dir = model_dir,name = "RNN_serving")
model_server.load()

In [ ]:
# test remotely

In [5]:
from mlrun import new_model_server, mount_v3io
import requests
import os

In [281]:
fn = new_model_server(name = 'RNN-serving',
                      model_class='RNN_Model_Serving')
fn.spec.description = "RNN based stock prediction"
fn.metadata.categories = ['RNN', 'Stocks', 'Predictions']
fn.metadata.labels = {'author': 'Daniel', 'framework': "tensorflow, keras"}
fn.spec.max_replicas = 1
fn.export("function.yaml")
fn.apply(mount_v3io())
fn.add_model('RNN_stocks', "/Users/test/demos/stock-analysis/models/RNN.h5")

> 2021-04-25 15:04:37,946 [info] function spec saved to path: function.yaml


In [ ]:
addr = fn.deploy()

> 2021-04-25 15:04:37,956 [info] Starting remote function deploy
2021-04-25 15:04:38  (info) Deploying function
2021-04-25 15:04:38  (info) Building
2021-04-25 15:04:38  (info) Staging files and preparing base images
2021-04-25 15:04:38  (info) Building processor image


In [256]:
# getting answer

In [257]:
data = [[[0.0, 0.0, 0.0003997438873852843, 0.0, 0.0031504590166214763, 0.07695351128043912],[0.0, 0.0076737889289563155, 0.0011658857834522252, 0.0034156784725392075, 0.0014412414129996165, 0.04148475898082488],[0.0, 0.005938872792230376, 0.0, 0.004308552624495143, 0.0052619477491432365, 0.030420689911339623],[0.0, 0.00974237019744173, 0.002864898969104024, 0.009126076646371328, 0.00754097274129828, 0.03420920993845116],[0.0, 0.012044527240189229, 0.006196233583000366, 0.01248470808067681, 0.009082756366275535, 0.03354111121910287],[0.0, 0.013579264661919943, 0.006129643489857672, 0.012247201899718174, 0.011462630413374697, 0.034077706918863734],[0.0, 0.01594821582320083, 0.009427632225050497, 0.01567375140495164, 0.014746872963329771, 0.041772700519581665],[0.00030003000301803695, 0.015347577857912631, 0.01465785196829128, 0.010008183299277662, 0.015383674399518488, 0.09121246281753899],[0.00030003000301803695, 0.019851802585909906, 0.01057694673341536, 0.01258648165342402, 0.015283029906551948, 0.04697070093018961],[0.00030003000301803695, 0.019718316169145544, 0.01062686388720635, 0.013909641632783631, 0.010724979922241862, 0.03015250378874856],[0.00030003000301803695, 0.015247488500415018, 0.007595438853113201, 0.011372653202875505, 0.008378858601442496, 0.024210897325231427],[0.00030003000301803695, 0.012878639159436767, 0.008361580749180142, 0.012077613789687636, 0.013104547126374655, 0.023022372892001874],[0.00030003000301803695, 0.017633036371026645, 0.010127184027974745, 0.01808246164906735, 0.013422334158536331, 0.030119960676463234],[0.00030003000301803695, 0.017983400032419605, 0.011592979391230274, 0.018353926865489978, 0.01575229508310738, 0.03006790591664354],[0.00040004000402404927, 0.026291122165628233, 0.018955200421947316, 0.01818423522181467, 0.015819289130762915, 0.07152689652172939],[0.00040004000402404927, 0.020318954134433742, 0.013358582670024766, 0.012077613789687636, 0.011462323570408328, 0.0456794077047931],[0.00040004000402404927, 0.01588142170466733, 0.007395465245156219, 0.007908106850034513, 0.005597122549398592, 0.031427718441502224],[0.00040004000402404927, 0.010042638269934567, 0.005296809836383476, 0.009621381602651136, 0.00738243720867815, 0.02567574365912337],[0.00040004000402404927, 0.011977834941958365, 0.006362759647989491, 0.010788412845792017, 0.008764355648181232, 0.022205280753766737],[0.00040004000402404927, 0.013178907231929493, 0.004497220397348678, 0.010177771409308312, 0.0074069846459871025, 0.025356051256133154],[0.00040004000402404927, 0.011810951465927255, 0.0022319372593224696, 0.0040711499771812765, 0.0, 0.031626735214947514],[0.0005000500050300616, 0.025123446934923743, 0.028016434638316157, 0.023612400680174672, 0.032778090757603406, 0.10153228594145776],[0.0005000500050300616, 0.03713457711584622, 0.03174761480386212, 0.03477405231513664, 0.030096896917537364, 0.047626905771733244],[0.0005000500050300616, 0.03456554905987275, 0.025784497378993576, 0.03300994254296863, 0.02869022647874786, 0.028768812095043804],[0.0005000500050300616, 0.03329870085439446, 0.026250831359521554, 0.03389163506129611, 0.028856842209482148, 0.025428460696670638],[0.0005000500050300616, 0.033264387412404006, 0.02518488154791565, 0.031652823528145024, 0.02952708952900418, 0.028119392147072767],[0.0005000500050300616, 0.03399840997415393, 0.029015896021043597, 0.03396659342006425, 0.03401824774664752, 0.033913843613464494],[0.0005000500050300616, 0.038369148285386845, 0.030614871570584734, 0.0350454139979145, 0.031102319037314974, 0.033925229639953834],[0.0006000600060360739, 0.04257300312058865, 0.04390696581173115, 0.03887905779413703, 0.04573187570650594, 0.09803619685873038],[0.0006000600060360739, 0.05008001039381649, 0.04099204802457179, 0.045291103478150685, 0.041925488708792535, 0.036871285277117145],[0.0006000600060360739, 0.04614302657184077, 0.0375107586207738, 0.04030399134624385, 0.03666589342239146, 0.04111875053787802],[0.0006000600060360739, 0.04090477928209357, 0.03571180946327568, 0.039964711592538005, 0.040218521286923936, 0.027399177725746942],[0.0006000600060360739, 0.044508199792612446, 0.03767738635002715, 0.04535891801543401, 0.042307610482901215, 0.030618589411891214],[0.0006000600060360739, 0.04657678106109786, 0.04117524702891262, 0.04627560490567262, 0.044793447634383865, 0.05405884268018613],[0.0006000600060360739, 0.04910742286297509, 0.047837916256705526, 0.04749626657677186, 0.05335661657762081, 0.06314168219836809],[0.007700770077008201, 0.030194810748639833, 0.06282892201923929, 0.03002444636418833, 0.05895373912701951, 0.2321397402054094],[0.007700770077008201, 0.06292535431428914, 0.05726493849616998, 0.060116707265659874, 0.05526702088618707, 0.078071078727893],[0.007700770077008201, 0.05958890663729899, 0.05200208452407595, 0.04851400230424485, 0.04286626924365666, 0.07557726468701688],[0.007700770077008201, 0.04721071426535017, 0.03784391241501628, 0.03962543183883216, 0.039346780419491134, 0.0617907570715274],[0.007700770077008201, 0.043530928527849455, 0.03890996389088652, 0.04125391253643351, 0.044340956539989596, 0.03782111959214908],[0.007700770077008201, 0.0485636006265614, 0.04137511897260526, 0.045935703950098494, 0.04155011747994375, 0.0446209643074681],[0.007700770077008201, 0.04580936144008119, 0.036544643116749764, 0.03738630970474699, 0.03525830245470285, 0.05051381704471508],[0.008000800080026238, 0.03993728276638431, 0.037731268410127417, 0.03160944293101264, 0.04028224234293831, 0.09797242089055323],[0.008000800080026238, 0.04313250568360927, 0.03691114278969587, 0.041932368510200546, 0.040620792415823725, 0.03983262723887141],[0.008000800080026238, 0.04514213299686476, 0.03984273351620693, 0.04647853084929898, 0.0439723358564007, 0.03439152856064892],[0.008000800080026238, 0.048335115867431555, 0.03857681009664404, 0.03775954849391683, 0.0353588446666806, 0.04126804866755406],[0.008000800080026238, 0.0397703992903532, 0.03627818107991432, 0.0411181799282222, 0.04165976269992355, 0.029229849676112238],[0.008000800080026238, 0.04600954015507641, 0.04194149058924401, 0.04400190253425518, 0.04668707786078907, 0.04065361967520816],[0.008000800080026238, 0.05101421167056164, 0.04513954335259052, 0.051906592774014215, 0.04899965101726622, 0.03669087617586497],[0.00810081008103225, 0.05958890663729899, 0.060596883095652365, 0.05923460061275354, 0.06327715652305188, 0.0879437184546251],[0.00810081008103225, 0.06756306545905066, 0.062162614430754326, 0.06873391604829482, 0.06699742292820654, 0.04326880002341804],[0.00810081008103225, 0.0711663841492669, 0.06549394904465067, 0.0727161306246723, 0.06872341461398956, 0.04025076154082211],[0.00810081008103225, 0.07293469734525959, 0.06572716686704694, 0.07304246867278985, 0.07101921368830555, 0.03401019316500588],[0.00810081008103225, 0.07523685438800698, 0.06839229555672255, 0.07562087056058087, 0.07413622682159382, 0.042899714001500375],[0.00810081008103225, 0.07830643105177104, 0.06949159124703208, 0.07365311010927356, 0.06820259981908527, 0.0531809708965342],[0.00810081008103225, 0.07233416120027403, 0.06372834576585618, 0.06798754200359969, 0.06445021718345156, 0.05271274214084092],[0.008200820082038263, 0.07760570372898534, 0.07595438853113234, 0.07419593700847404, 0.0738178261034993, 0.1051673434581077],[0.008200820082038263, 0.07803945821824254, 0.07592104265242872, 0.07908127556763367, 0.0799008856306257, 0.06479979642881596],[0.008200820082038263, 0.0840451251290063, 0.07642072251166032, 0.08345774626305669, 0.07933107824209251, 0.038738024827896046],[0.008200820082038263, 0.08354457652121561, 0.07695374824959555, 0.08169353295724402, 0.07758759650722691, 0.037608309576979866]]]

In [258]:
import json

event_data = {'inputs':X_test[:5].tolist()}

resp = requests.put(addr+"/RNN_stocks/predict", json = event_data)
s = json.loads(json.loads(resp.text))